<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/modelToDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##モデルをDB化

In [0]:
!pip install mecab-python3

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.00161192days (139.27sec)


In [0]:
import IPython
#bz2ファイル解凍
#!bunzip2 -k -q "./drive/My Drive/NLP/20170201.tar.bz2"

#tarファイル解凍
!tar -xvf "./drive/My Drive/NLP/20170201.tar"

from gensim.models import KeyedVectors

#解凍時のみ
model_dir = 'entity_vector/entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True,limit=500000)

#解凍後確認
results = model.most_similar(positive=['柔らかい'],topn=5)
for result in results:
  print(result)

entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


##語彙リスト作成

In [0]:
import re
import pandas as pd

cnt = 1
model_vocab_list = []

for words, v in model.vocab.items():

  #固有名詞用の括弧を削除
  #words = words.replace('[', '').replace(']', '')

  #-----------------------------------------------------  

  # 半角記号+半角数字
  #p = re.compile("[!-@[-`{-~]")
  
  # 半角記号+半角数字+半角英字
  #((ほぼ)日本語のみのDB作成用)
  p = re.compile("[!-~]")

  # 先頭文字
  top_char = words[0:1]

  #-----------------------------------------------------  

#   【M1】dfに格納した後で補足情報を削除する為、ココでは補足情報を残しておく
#   #補足情報の_を削除
#   words = words.replace("_","")
#   #括弧()文字を抽出
#   regex = re.compile(".*?\((.*?)\)")
#   #括弧文字をlist型で返却
#   ret_list = re.findall(regex, words)

  #-----------------------------------------------------  

#   #除外文字なし
#   if p.match(top_char) is None:
#     #括弧文字がある場合は無効化
#     if len(ret_list) > 0: 
#       words = words.replace("(" + ret_list[0] + ")",'')
#       model_vocab_list.append(words)
#     else:
#       model_vocab_list.append(words)

  #【M1】
  model_vocab_list.append(words)
  #【M1】
  
  if cnt % 50000 == 0:
    print(cnt,"件 完了");
    
  cnt = cnt + 1

#重複削除
model_vocab_uni_list= list(set(model_vocab_list))
  
#保存
drive.mount('/content/drive')
df = pd.DataFrame(model_vocab_uni_list)
df.to_csv('./drive/My Drive/NLP/model_vocab_list.csv', header=False, index=False)

50000 件 完了
100000 件 完了
150000 件 完了
200000 件 完了
250000 件 完了
300000 件 完了
350000 件 完了
400000 件 完了
450000 件 完了
500000 件 完了
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##語彙リスト読み込み

In [0]:
import pandas as pd

drive.mount('/content/drive')
csv_dir = './drive/My Drive/NLP/model_vocab_list.csv'
df = pd.read_csv(csv_dir,names=['words'])
df = df.sort_values('words', ascending=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##(関数)対義語-対応list作成

In [0]:
import MeCab
#括弧記号無効化
import lib_delete_brackets

def search_similar_texts(p_words):
  pass

def wordRevChange(words,gyaku,model,tokenizer):

  no_list = ['混一色','ぎゃく','必然的','意図的','結果的','下向き','場合','横方向','かえって']

  #**************************************************************************
  #補足情報有無判定
  #**************************************************************************
  if '_(' in words:
    #補足情報あり･･･①
    info_chk_flg = 0
  else:
    #補足情報なし and 記事名･･･②
    if '[' in words:
      info_chk_flg = 1
    else:
      #ノーマル語彙･･･③
      info_chk_flg = 2
  
  #**************************************************************************
  #①そのまま対義語化
  #**************************************************************************
  if info_chk_flg == 0:
    asitis_cng_words1 = ''
    asitis_cng_words2 = ''
    asitis_cng_words3 = ''

    #--------------------------------------------------------
    #★対義語リスト取得
    #--------------------------------------------------------
    rvs_wd_list = model.most_similar(positive=[words,gyaku])

    index = 0
    rvs_wd_rt_list = []

    for index in range(len(rvs_wd_list)):
      wd = rvs_wd_list[index][0]
      if str(wd) not in no_list:

        #対義語候補格納
        rvs_wd_rt_list.append(lib_delete_brackets.delete_brackets(wd))

    result_list = []

    asitis_cng_words1 = lib_delete_brackets.delete_brackets(rvs_wd_rt_list[0])
    asitis_cng_words2 = lib_delete_brackets.delete_brackets(rvs_wd_rt_list[1])
    asitis_cng_words3 = lib_delete_brackets.delete_brackets(rvs_wd_rt_list[2])

    asitis_cng_words1 = asitis_cng_words1.replace('_',"").replace('[',"").replace(']',"")
    asitis_cng_words2 = asitis_cng_words2.replace('_',"").replace('[',"").replace(']',"")
    asitis_cng_words3 = asitis_cng_words3.replace('_',"").replace('[',"").replace(']',"")

    #返却値書き込み
    result_list.append(words.replace('[',"").replace(']',""))
    result_list.append(asitis_cng_words1)
    result_list.append(asitis_cng_words2)
    result_list.append(asitis_cng_words3)
    result_list.append(info_chk_flg)

    return result_list

  #**************************************************************************
  #②記事名
  #**************************************************************************
  #---------------------------------------------
  #②-1 記事名をそのまま対義語化
  #---------------------------------------------
  if info_chk_flg == 1:

    result_list = []
    rvs_wd_rt_list = []

    asitis_cng_words1 = ''
    asitis_cng_words2 = ''
    asitis_cng_words3 = ''

    #--------------------------------------------------------
    #★対義語リスト取得
    #--------------------------------------------------------
    rvs_wd_list = model.most_similar(positive=[words,gyaku])

    index = 0

    for index in range(len(rvs_wd_list)):
      wd = rvs_wd_list[index][0]
      if str(wd) not in no_list:

        #対義語候補格納
        rvs_wd_rt_list.append(lib_delete_brackets.delete_brackets(wd))
    
    #②-1は「1候補」のみセット
    asitis_cng_words1 = lib_delete_brackets.delete_brackets(rvs_wd_rt_list[0])
    #asitis_cng_words2 = lib_delete_brackets.delete_brackets(rvs_wd_rt_list[1])

    asitis_cng_words1 = asitis_cng_words1.replace('_',"").replace('[',"").replace(']',"")
    #asitis_cng_words2 = asitis_cng_words2.replace('_',"").replace('[',"").replace(']',"")

    #返却値書き込み
    result_list.append(words.replace('[',"").replace(']',""))
    result_list.append(asitis_cng_words1)
    #result_list.append(asitis_cng_words2)
    
    #この時点ではreturnなし

    #---------------------------------------------
    #②-2 形態素分析後に強制的に対義語化
    #---------------------------------------------
    node = tokenizer.parseToNode(words)

    word_cng_str1 = ''
    word_cng_str2 = ''
    
    while node:
      cut_wd = node.surface
      
      #特定固有名詞の囲み文字は処理対象外
      if(cut_wd != "[" and cut_wd != "]" and cut_wd != "(" and cut_wd != ")" and cut_wd != "_"):
        if node.feature.split(",")[0] == u"名詞":
          try:
            #--------------------------------------------------------
            #★対義語リスト取得
            #--------------------------------------------------------
            rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
            for i in range(4):
              wd1 = rvs_wd_list[i][0].replace('[',"").replace(']',"")
              if (cut_wd != wd1 and
                 str(wd1) not in no_list):
                #◆結合1
                word_cng_str1 = word_cng_str1 + wd1

                #◆結合2
                for ii in range(4):
                  wd2 = rvs_wd_list[i+ii+1][0].replace('[',"").replace(']',"")
                  if str(wd2) not in no_list:
                    word_cng_str2 = word_cng_str2 + wd2
                    break

                break
          except KeyError as error:
            #辞書に登録の無い単語の場合
            word_cng_str1 = word_cng_str1 + cut_wd
            word_cng_str2 = word_cng_str2 + cut_wd

        elif (node.feature.split(",")[0] == u"動詞" or
          node.feature.split(",")[0] == u"形容詞" or
          node.feature.split(",")[0] == u"副詞" or
          node.feature.split(",")[0] == u"感動詞"):
          cut_wd = node.feature.split(",")[6]

          try:
            #--------------------------------------------------------
            #★対義語リスト取得
            #--------------------------------------------------------
            rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
            for i in range(4):
              wd1 = rvs_wd_list[i][0].replace('[',"").replace(']',"")
              if (cut_wd != wd1 and
                 str(wd1) not in no_list):
                #◆結合1
                word_cng_str1 = word_cng_str1 + wd1

                #◆結合2
                for ii in range(4):
                  wd2 = rvs_wd_list[i+ii+1][0].replace('[',"").replace(']',"")
                  if str(wd2) not in no_list:
                    word_cng_str2 = word_cng_str2 + wd2
                    break
                    
                break
          except KeyError as error:
            #辞書に登録の無い単語の場合
            word_cng_str1 = word_cng_str1 + cut_wd
            word_cng_str2 = word_cng_str2 + cut_wd

        else:
          #◆結合
          word_cng_str1 = word_cng_str1 + cut_wd.replace('[',"").replace(']',"")
          word_cng_str2 = word_cng_str2 + cut_wd.replace('[',"").replace(']',"")

      node = node.next

    #返却値書き込み
    #②-2は「2候補」セット
    asitis_cng_words2 = word_cng_str1.replace('_',"")
    asitis_cng_words3 = word_cng_str2.replace('_',"")

    result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words2))
    result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words3))
    result_list.append(info_chk_flg)

    return result_list

  #**************************************************************************
  #③ノーマル語彙
  #**************************************************************************
  #---------------------------------------------
  #形態素分析後に強制的に対義語化
  #---------------------------------------------
  if info_chk_flg == 2:
    
    word_cng_str1 = ''
    word_cng_str2 = ''
    word_cng_str3 = ''

    node = tokenizer.parseToNode(words)

    while node:
      cut_wd = node.surface
      
      #特定固有名詞は処理対象外
      if(cut_wd != "[" and cut_wd != "]" and cut_wd != "(" and cut_wd != ")" and cut_wd != "_"):
        if node.feature.split(",")[0] == u"名詞":
          try:
            #--------------------------------------------------------
            #★対義語リスト取得
            #--------------------------------------------------------
            rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
            for i in range(4):
              wd1 = rvs_wd_list[i][0].replace('[',"").replace(']',"")
              if (cut_wd != wd1 and
                 str(wd1) not in no_list):
                #◆結合1
                word_cng_str1 = word_cng_str1 + wd1
                
                #◆結合2
                for ii in range(4):
                  wd2 = rvs_wd_list[i+ii+1][0].replace('[',"").replace(']',"")
                  if str(wd2) not in no_list:
                    word_cng_str2 = word_cng_str2 + wd2
                    ii_save = i+ii+1
                    
                    #◆結合3
                    for iii in range(4):
                      wd3 = rvs_wd_list[ii_save+iii+1][0].replace('[',"").replace(']',"")
                      if str(wd3) not in no_list:
                        word_cng_str3 = word_cng_str3 + wd3
                        break
                    break
                break
          except KeyError as error:
            #辞書に登録の無い単語の場合
            word_cng_str1 = word_cng_str1 + cut_wd
            word_cng_str2 = word_cng_str2 + cut_wd
            word_cng_str3 = word_cng_str3 + cut_wd
          
        elif (node.feature.split(",")[0] == u"動詞" or
          node.feature.split(",")[0] == u"形容詞" or
          node.feature.split(",")[0] == u"副詞" or
          node.feature.split(",")[0] == u"感動詞"):
          cut_wd = node.feature.split(",")[6]

          try:
            #--------------------------------------------------------
            #★対義語リスト取得
            #--------------------------------------------------------
            rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
            for i in range(4):
              wd1 = rvs_wd_list[i][0].replace('[',"").replace(']',"")
              if (cut_wd != wd1 and
                 str(wd1) not in no_list):
                #◆結合1
                word_cng_str1 = word_cng_str1 + wd1
                
                #◆結合2
                for ii in range(4):
                  wd2 = rvs_wd_list[i+ii+1][0].replace('[',"").replace(']',"")
                  if str(wd2) not in no_list:
                    word_cng_str2 = word_cng_str2 + wd2
                    ii_save = i+ii+1

                    #◆結合3
                    for iii in range(4):
                      wd3 = rvs_wd_list[ii_save+iii+1][0].replace('[',"").replace(']',"")
                      if str(wd3) not in no_list:
                        word_cng_str3 = word_cng_str3 + wd3
                        break
                    break
                break
          except KeyError as error:
            #辞書に登録の無い単語の場合
            word_cng_str1 = word_cng_str1 + cut_wd
            word_cng_str2 = word_cng_str2 + cut_wd
            word_cng_str3 = word_cng_str3 + cut_wd

        else:
          #◆結合
          word_cng_str1 = word_cng_str1 + cut_wd.replace('[',"").replace(']',"")
          word_cng_str2 = word_cng_str2 + cut_wd.replace('[',"").replace(']',"")
          word_cng_str3 = word_cng_str3 + cut_wd.replace('[',"").replace(']',"")

      node = node.next

    result_list = []

    #返却値書き込み
    asitis_cng_words1 = word_cng_str1.replace('_',"")
    asitis_cng_words2 = word_cng_str2.replace('_',"")
    asitis_cng_words3 = word_cng_str3.replace('_',"")

    result_list.append(words.replace('(',"").replace(')',""))
    result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words1))
    result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words2))
    result_list.append(lib_delete_brackets.delete_brackets(asitis_cng_words3))
    result_list.append(info_chk_flg)

    return result_list


##(関数呼出)対義語-対応list作成

In [0]:
import re
import pandas as pd
import MeCab
#対義語生成
import lib_delete_brackets
#import lib_wordRevChange as lw

tokenizer = MeCab.Tagger("-Ochasen")
owkt = MeCab.Tagger("-Owakati")        

cnt = 1
save_list = []
del_char_list = ["も","が","は","の","で","に","を","と","や","か"]

jpn_start_flg = 0

#列[words]をnumpyに変換
val = df.words.values

for idx in range(df.shape[0]):

  words = str(val[idx])
  
  #日本語単語の開始判定
  if words[0:2] == "嘶き":
    jpn_start_flg = 1

#   #日本語単語の終了判定
#   if words[0:1] == "龍":
#     break
  
  if jpn_start_flg == 1:

    #語彙数チェック
    cnt_words_str = lib_delete_brackets.delete_brackets(words.replace("[","").replace("]","")).replace("_","")

    #10文字以下が対象
    if len(cnt_words_str) <= 10:
      result_owkt = owkt.parse(cnt_words_str)
      owkt_list = result_owkt.split(' ')
      goi_cnt = len(owkt_list)-1
    else:
      goi_cnt = 99
      
#     for del_char in del_char_list:
#       try:
#         owkt_list.remove(del_char)
#       except ValueError as error:
#         pass

    #語彙に含まれる形態素数が5以下のみ処理対象
    if goi_cnt <= 5:
      if '#' not in words:
        if ':' not in words:
          if '・' not in words:
            if words.isdecimal() is False:
              gyaku = u"逆"
              rev_list = wordRevChange(words,gyaku,model,tokenizer)

              #返却値ありのみ格納
              if rev_list is not None:
                save_list.append(rev_list)

  if cnt % 100 == 0:
    print(cnt,"件 終了")

    if cnt % 1500 == 0:
      #Gドライブ再マウント
      drive.mount('/content/drive')  
      #保存
      df2 = pd.DataFrame(save_list)
      df2.to_csv('./drive/My Drive/NLP/modelToCsv_4.csv', header=False, index=False)
    
  cnt = cnt + 1
  
#Gドライブ再マウント
drive.mount('/content/drive')  
#保存
df2 = pd.DataFrame(save_list)
df2.to_csv('./drive/My Drive/NLP/modelToCsv_4.csv', header=False, index=False)


##以下、検証用

##(取得)対義語関数

In [0]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 5.9MB/s 


In [0]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
csv_dir = './drive/My Drive/NLP/modelToCsv.csv'
df_ant = pd.read_csv(csv_dir,names=['words','ant1','ant2','ant3','flg'])
df_ant = df_ant.sort_values('words', ascending=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import MeCab
import pandas as pd
import numpy as np

def get_ant_word(words):

  word_cng_list = []

  #-------------------------------------------------
  # そのまま対義語化
  #-------------------------------------------------
  dfTolist = df_ant[df_ant["words"] == words].values.tolist()
  ant_list = [flatten for inner in dfTolist for flatten in inner]
  
  #-------------------------------------------------
  # 形態素分析後に対義語化
  #-------------------------------------------------
  if len(dfTolist) == 0:
    tokenizer = MeCab.Tagger("-Ochasen")
    node = tokenizer.parseToNode(words)
    ant_word = ''
    rvs_wd = ''
    
    while node:
      #分かち書きの単語を取得
      cut_wd = node.surface
      
      #数字はそのまま
      if cut_wd.isnumeric():
        ant_word = ant_word + cut_wd 
      else:      
        if cut_wd != np.nan and cut_wd != '':
          if node.feature.split(",")[0] == u"名詞":
            try:
              rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][1]
              if rvs_wd is np.nan or rvs_wd == cut_wd:
                rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][2]
                if  rvs_wd is np.nan or rvs_wd == cut_wd:
                  rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][3]

              ant_word = ant_word + str(rvs_wd)

            except IndexError as error:
              #辞書に登録の無い単語の場合
              ant_word = ant_word + str(cut_wd)

          elif (node.feature.split(",")[0] == u"動詞" or
            node.feature.split(",")[0] == u"形容詞" or
            node.feature.split(",")[0] == u"副詞" or
            node.feature.split(",")[0] == u"感動詞"):

            #分かち書きの単語を取得
            cut_wd = node.feature.split(",")[6]

            try:
              rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][1]
              if rvs_wd is np.nan or rvs_wd == cut_wd:
                rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][2]
                if rvs_wd is np.nan or rvs_wd == cut_wd:
                  rvs_wd = df_ant[df_ant["words"] == cut_wd].values[0][3]

              ant_word = ant_word + str(rvs_wd)

            except IndexError as error:
              #辞書に登録の無い単語の場合
              ant_word = ant_word + str(cut_wd)

          else:
            #◆結合
            ant_word = ant_word + str(cut_wd)

      node = node.next

    word_cng_list.append(ant_word)
  
    return word_cng_list
  else:
    return ant_list


In [0]:
print(get_ant_word("全国高等学校バスケットボール選抜優勝大会"))


['全国高等学校野球選抜優勝大会']


In [0]:
print(df_ant[df_ant['words'].str.contains('叫ぶ',na=False)])
print(df_ant[df_ant['words']=="叫ぶ"])

         words  info ant1            ant2     ant3
111040  叫ぶ詩人の会   NaN  夜光虫  ハイテクノロジー・スーサイド  怒る哲学者の気
Empty DataFrame
Columns: [words, info, ant1, ant2, ant3]
Index: []


In [0]:
import MeCab
no_list = ['ぎゃく','必然的','意図的','結果的','下向き','場合','横方向','かえって']
words = "あいさつ"
gyaku = "逆"

tokenizer = MeCab.Tagger("-Ochasen")

word_cng_str1 = ''
word_cng_str2 = ''
word_cng_str3 = ''

node = tokenizer.parseToNode(words)

while node:
  cut_wd = node.surface

  #特定固有名詞は処理対象外
  if(cut_wd != "[" and cut_wd != "]" and cut_wd != "(" and cut_wd != ")" and cut_wd != "_"):
    if node.feature.split(",")[0] == u"名詞":
      try:
        #--------------------------------------------------------
        #★対義語リスト取得
        #--------------------------------------------------------
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(4):
          wd1 = rvs_wd_list[i][0].replace('[',"").replace(']',"")
          if (cut_wd != wd1 and
             str(wd1) not in no_list):
            #◆結合1
            word_cng_str1 = word_cng_str1 + wd1
            print("wd1:",wd1)

            #◆結合2
            for ii in range(4):
              wd2 = rvs_wd_list[i+ii+1][0].replace('[',"").replace(']',"")
              print("wd2:",i+ii+1,wd2)
              if str(wd2) not in no_list:
                word_cng_str2 = word_cng_str2 + wd2
                ii_save = i+ii+1

                #◆結合3
                for iii in range(4):
                  wd3 = rvs_wd_list[ii_save+iii+1][0].replace('[',"").replace(']',"")
                  print("wd3:",ii_save+iii+1,wd3)
                  if str(wd3) not in no_list:
                    word_cng_str3 = word_cng_str3 + wd3
                    break
                break
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        word_cng_str1 = word_cng_str1 + cut_wd
        word_cng_str2 = word_cng_str2 + cut_wd
        word_cng_str3 = word_cng_str3 + cut_wd

    elif (node.feature.split(",")[0] == u"動詞" or
      node.feature.split(",")[0] == u"形容詞" or
      node.feature.split(",")[0] == u"副詞" or
      node.feature.split(",")[0] == u"感動詞"):
      cut_wd = node.feature.split(",")[6]

      try:
        #--------------------------------------------------------
        #★対義語リスト取得
        #--------------------------------------------------------
        rvs_wd_list = model.most_similar(positive=[cut_wd,gyaku])
        for i in range(4):
          wd1 = rvs_wd_list[i][0].replace('[',"").replace(']',"")
          if (cut_wd != wd1 and
             str(wd1) not in no_list):
            #◆結合1
            word_cng_str1 = word_cng_str1 + wd1
            print("wd1:",wd1)

            #◆結合2
            for ii in range(4):
              wd2 = rvs_wd_list[i+ii+1][0].replace('[',"").replace(']',"")
              print("wd2:",i+ii+1,wd2)
              if str(wd2) not in no_list:
                word_cng_str2 = word_cng_str2 + wd2
                ii_save = i+ii+1

                #◆結合3
                for iii in range(4):
                  wd3 = rvs_wd_list[ii_save+iii+1][0].replace('[',"").replace(']',"")
                  print("wd3:",ii_save+iii+1,wd3)
                  if str(wd3) not in no_list:
                    word_cng_str3 = word_cng_str3 + wd3
                    break
                break
            break
      except KeyError as error:
        #辞書に登録の無い単語の場合
        word_cng_str1 = word_cng_str1 + cut_wd
        word_cng_str2 = word_cng_str2 + cut_wd
        word_cng_str3 = word_cng_str3 + cut_wd

    else:
      #◆結合
      word_cng_str1 = word_cng_str1 + cut_wd.replace('[',"").replace(']',"")
      word_cng_str2 = word_cng_str2 + cut_wd.replace('[',"").replace(']',"")
      word_cng_str3 = word_cng_str3 + cut_wd.replace('[',"").replace(']',"")

  node = node.next

print(word_cng_str1)
print(word_cng_str2)
print(word_cng_str3)

wd1: 挨拶
wd2: 1 必ず
wd3: 2 乾杯
挨拶
必ず
乾杯


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
a = model["日本"]
b = model["高齢者"]

vector = a - b

word = model.most_similar( [ vector ], [], 10)
word